In [196]:
import pandas as pd
import numpy as np

# read dataset into pandas dataframe
df_items = pd.read_csv('items-Copy1.csv')

# replace field that's entirely space (or empty) with NaN
df_items = df_items.replace(np.nan, '', regex=True)

In [197]:
# Take rows with tags 'black', 'white', both or neither and label them as such
# 'black' = 0, 'white' = 1, both = 2, neither = 3
labels_list_word = []
labels_list_numerical = []
descriptions_list = []

for i, row in df_items.iterrows():
    row['tag'] = row['tag'].lower()
    if 'anger' in row['tag'] and not 'fear' in row['tag']: #and 'white' in row['tag']:
        labels_list_word.append('anger') # black
        descriptions_list.append(row['description'])
        labels_list_numerical.append(0) # 0
    elif not 'anger' in row['tag'] and 'fear' in row['tag']:
        labels_list_word.append('fear') # white
        descriptions_list.append(row['description'])
        labels_list_numerical.append(1) # 1
    elif 'anger' in row['tag'] and 'fear' in row['tag']:
        labels_list_word.append('both') # both
        descriptions_list.append(row['description'])
        labels_list_numerical.append(2) # 2
    else:
        labels_list_word.append('neither') # neither
        descriptions_list.append(row['description'])
        labels_list_numerical.append(3) # 3

df = pd.DataFrame()
df['label word'] = labels_list_word # append labels_list_word to df with column header 'label word'
df['label numerical'] = labels_list_numerical # append labels_list_numerical to df with column header 'label numerical'
df['description'] = descriptions_list # append descriptions_list to df with column header 'description'
df

,label word,label numerical,description
0,fear,1,Black American racial experience is real. We support the 2nd ammendment\nfor our safety.
1,anger,0,The best costume for Halloween worth posting. Power to the People!
2,anger,0,"Keep on whining and crying for your president, ooh, who wasn't elected.\nImma take care of my kids.\n\nI'll keep grinding for them, to make sure the get the best education.\nTo ensure they dress well and eat healthy food.\nAnd I'm gon tell you to do the same thing.\n\nDon't waste time on an ""afterparty"" you wouldn't have been invited to!\n\n#word #wakeup #organize #joshuabeal #altonsterling #TerenceCrutCher\n#westandtogether #blacklivesmatter #unite4justice #hovem ber8\n#electi0h52016 #black #melanin #blackisbeautiful #blacklivesmatter\n#blackgirlmagic #USA #blackahdproud #democrat #republioan\n#Africanamerican #unite4justice #vote #breakingnews"
3,anger,0,"End the whining and crying, end the riots with these same white\npeople(democrats), who will not march with you when your son, daughter or\nloved one gets shot for being Black.\n\nI opted out of the elections and i had many Black people all up in my case,\ntelling me how i was wrong and how i needed to make my vote count for Mrs.\nClinton, all just to make sure Trump doesn't become president.\n\nWell now all i want to say is get over Trump and Clinton, they are rich white\npeople, the elite who were born into and are a part of the system.\n\nI dream of liberation, not staying a slave under Hillary or Trump, so forget\nyour Childish rioting, no one has shot!\n\n#wakeup #organize #joshuabeal #altonsterling #TerenceCrutCher\n#westandtogether #blacklivesmatter #unite4justice #novem ber8\n#elections2016 #black #melanin #blackisbeautiful #blacklivesmatter\n#blackgirlmagic #USA #blackandproud #democrat #republioan\n#Africanamerican #unite4justice #vote #breakingnews"
4,neither,3,Black girls are the definition of national greatness.
...,...,...,...
3007,neither,3,"New ideas, old values. Like us!"
3008,neither,3,Secured borders are a national priority. We need to protect America now\nmore than ever.
3009,neither,3,Secured borders should be a top priority. We need to protect America now\nmore than ever.
3010,neither,3,Bernie for president!


In [198]:
pd.set_option('display.max_colwidth', -1) # set display options to display entire non-turnicated contents
print(df[['label numerical', 'description']].head(10)) 

   label numerical  \
0  1                 
1  0                 
2  0                 
3  0                 
4  3                 
5  3                 
6  3                 
7  0                 
8  0                 
9  0                 

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [199]:
'''df = df[['label word', 'description']]

df_anger = df.loc[df['label word'] == "anger"]
df_fear = df.loc[df['label word'] == "fear"]
df_both = df.loc[df['label word'] == "both"]
df_neither = df.loc[df['label word'] == "neither"]

df_anger'''

'df = df[[\'label word\', \'description\']]\n\ndf_anger = df.loc[df[\'label word\'] == "anger"]\ndf_fear = df.loc[df[\'label word\'] == "fear"]\ndf_both = df.loc[df[\'label word\'] == "both"]\ndf_neither = df.loc[df[\'label word\'] == "neither"]\n\ndf_anger'

In [200]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df['description'], 
                                                    df['label numerical'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 3012
Number of rows in the training set: 2259
Number of rows in the test set: 753


In [201]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate Countvectorizer method
count_vector = CountVectorizer()

# fit training data and return matrix
training_data = count_vector.fit_transform(X_train)

# transform testing data and return matrix
testing_data = count_vector.transform(X_test)

In [202]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, Y_train)

predictions = naive_bayes.predict(testing_data)

In [213]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

print(classification_report(Y_test, predictions, target_names = ['anger', 'fear', 'both', 'neither']))

print('Accuracy score: ', format(accuracy_score(predictions, Y_test)))
print('Precision score: ', format(precision_score(predictions, Y_test, average='macro')))
print('Recall score: ', format(recall_score(predictions, Y_test, average='macro')))
print('F1 score: ', format(f1_score(predictions, Y_test, average='macro')))

              precision    recall  f1-score   support

       anger       0.44      0.43      0.43       159
        fear       0.00      0.00      0.00        32
        both       0.47      0.20      0.28        46
     neither       0.78      0.87      0.82       516

    accuracy                           0.70       753
   macro avg       0.42      0.37      0.38       753
weighted avg       0.66      0.70      0.67       753

Accuracy score:  0.701195219123506
Precision score:  0.3743390344099561
Recall score:  0.4228307690503209
F1 score:  0.38344709706720004
